# Summarize CHM results from DINOv3 - create area summary for height bins

### Note: you want to update the 002m CHMs with the new cloudmasks that were used in the `dm_10m` versions so you get accurate `total area` estimates for each `height_bin`

In [10]:
import rasterio
import numpy as np
from multiprocessing import Pool
import pandas as pd
import glob

In [11]:
def compute_height_class_areas(filepath, height_bins=None):
    """
    Compute area for each height class in a single raster.
    
    Parameters:
    -----------
    filepath : str
        Path to GeoTIFF
    height_bins : list of tuples
        Height ranges, e.g., [(0.25, 5), (5, 10), (10, 20)]
    
    Returns:
    --------
    dict with areas per height class
    """
    if height_bins is None:
        height_bins = [(0,0.25), (0.25, 5), (5, 10), (10, 20), (20, 50)]
    
    try:
        with rasterio.open(filepath) as src:
            data = src.read(1)
            transform = src.transform
            
            # Get pixel area in m²
            pixel_width = abs(transform[0])   # meters
            pixel_height = abs(transform[4])  # meters
            pixel_area_m2 = pixel_width * pixel_height
            pixel_area_km2 = pixel_area_m2 / 1e6
            
            # Mask nodata
            if src.nodata is not None:
                valid_mask = data != src.nodata
            else:
                valid_mask = np.ones_like(data, dtype=bool)
            
            # Calculate area for each height class
            results = {'file': filepath}
            
            for min_h, max_h in height_bins:
                class_name = f'{min_h}-{max_h}m'
                mask = (data >= min_h) & (data < max_h) & valid_mask
                n_pixels = np.sum(mask)
                area_km2 = n_pixels * pixel_area_km2
                results[f'area_{class_name}'] = area_km2
                results[f'n_pixels_{class_name}'] = n_pixels
            
            # IMPORTANT: Total valid area for this file
            results['total_valid_area_km2'] = np.sum(valid_mask) * pixel_area_km2
            results['n_valid_pixels'] = np.sum(valid_mask)
            
            return results
            
    except Exception as e:
        print(f"Error processing {filepath}: {e}")
        return None

In [4]:
import re

In [41]:
files_chm = glob.glob('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/002m/*-sr-02m.chm.tif')
# Filter for July (07) and August (08) in YYYYMMDD format
# Pattern matches: WV02_20230715_*, WV03_20190828_*, etc.
files_chm_jujlyaug = [f for f in files_chm if re.search(r'_\d{4}(07|08)\d{2}_', f)]

print(f"Found {len(files_chm_jujlyaug)} files from July and August (out of {len(files_chm)} total)")

Found 1288 files from July and August (out of 3219 total)


In [42]:
files_chm_jujlyaug = files_chm_jujlyaug[0:]

In [43]:
height_bins = [(0,0.25), (0.25, 1.5), (1.5,2.5), (2.5,3.5), (3.5, 5), (5, 10), (10, 20), (20, 50)]

In [44]:
%%time
from joblib import Parallel, delayed

# Much more Jupyter-friendly
results = Parallel(n_jobs=35, verbose=10)(
    delayed(compute_height_class_areas)(f, height_bins) 
    for f in files_chm_jujlyaug
)

area_stats_df = pd.DataFrame([r for r in results if r is not None])

[Parallel(n_jobs=35)]: Using backend LokyBackend with 35 concurrent workers.
[Parallel(n_jobs=35)]: Done   2 tasks      | elapsed:   10.7s
[Parallel(n_jobs=35)]: Done  15 tasks      | elapsed:   18.7s
[Parallel(n_jobs=35)]: Done  28 tasks      | elapsed:   26.6s
/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=35)]: Done  43 tasks      | elapsed:   30.6s
[Parallel(n_jobs=35)]: Done  58 tasks      | elapsed:   39.1s
[Parallel(n_jobs=35)]: Done  75 tasks      | elapsed:   44.6s
[Parallel(n_jobs=35)]: Done  92 tasks      | elapsed:   54.2s
[Parallel(n_jobs=35)]: Done 111 tasks      | elapsed:  1.1min
[Parallel(n_jobs=35)]: Done 130 tasks      | elapsed:  1.3min
[Parallel(n_jobs=35)]: Done 151 tasks      | elapsed:  1.4min
[Parallel(

CPU times: user 3.08 s, sys: 1.22 s, total: 4.3 s
Wall time: 9min


[Parallel(n_jobs=35)]: Done 1288 out of 1288 | elapsed:  9.0min finished


In [39]:
area_stats_df['n_pixels_0-0.01m'].sum()

4651345205

In [38]:
area_stats_df['n_pixels_0.01-1.01m'].sum()

1174991479

In [35]:
area_stats_df['n_pixels_0.01-1.2m'].sum()

1174991479

In [32]:
area_stats_df['n_pixels_0.01-1.0m'].sum()

0

In [28]:
area_stats_df['n_pixels_1.0-1.5m'].sum()

1174991479

In [27]:
area_stats_df['n_pixels_0.1-1.0m'].sum()

0

In [24]:
area_stats_df['n_pixels_0.5-1.0m'].sum()

0

In [45]:
# Save detailed results
area_stats_df.to_csv('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/chm_002m_height_class_areas_by_file.csv', index=False)